In [3]:
from collections import defaultdict

def viterbi_pcfg(words, pcfg_rules):
    n = len(words)
    table = [[defaultdict(lambda: (0.0, None)) for _ in range(n)] for _ in range(n)]

    # Initialization
    for i, word in enumerate(words):
        for nt, (prob, terminals) in pcfg_rules.items():
            if word in terminals:
                table[i][i][nt] = (prob, None)

    # Viterbi Algorithm
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            for k in range(i, j):
                for A, (prob_A, _) in pcfg_rules.items():
                    for B, (prob_B, _) in pcfg_rules.items():
                        for C in table[i][k]:
                            for D in table[k + 1][j]:
                                prob = prob_A * prob_B * pcfg_rules[A][1].count(C) * pcfg_rules[B][1].count(D)
                                if prob > table[i][j][A][0]:
                                    table[i][j][A] = (prob, (C, D, k))

    # Reconstruct the most probable parse tree
    def reconstruct_tree(i, j, nt):
        if table[i][j][nt][1] is None:
            return (nt, words[i])
        else:
            C, D, k = table[i][j][nt][1]
            left_subtree = reconstruct_tree(i, k, C)
            right_subtree = reconstruct_tree(k + 1, j, D)
            return (nt, left_subtree, right_subtree)

    # Get the most probable parse tree and its probability
    parse_tree = reconstruct_tree(0, n - 1, 'S')
    parse_probability = table[0][-1]['S'][0]

    return parse_tree, parse_probability

# Different PCFG rules
pcfg_rules = {
    'S': (1.0, ['NP', 'VP']),
    'NP': (0.6, ['Det', 'N']),
    'VP': (0.7, ['V', 'NP']),
    'Det': (1.0, ['the', 'a']),
    'N': (0.5, ['cat', 'dog', 'bat']),
    'V': (0.8, ['chased', 'caught'])
}

# Different input sentence
words = ['the', 'cat', 'chased', 'a', 'bat']

# Call Viterbi PCFG algorithm to get the most probable parse tree and its probability
parse_tree, parse_probability = viterbi_pcfg(words, pcfg_rules)

# Print the most probable parse tree and its probability
print(f'Most Probable Parse Tree: {parse_tree}')
print(f'Parse Probability: {parse_probability}')


Most Probable Parse Tree: ('S', ('NP', ('Det', 'the'), ('N', 'cat')), ('NP', ('Det', 'chased'), ('N', 'bat')))
Parse Probability: 1.0
